### Data Preperation
### Term Project - MileStone 2
### Submitter - Himanshu Singh
### Cleaning/Formatting Flat File Source

In [18]:
# We got the Latitude and Longitude level info in covid_dataset. It also has data from Jan 22 to March 24 2021.
# We got now Weekly Covid Cases and Weekly Covid Deaths information from https://ourworldindata.org/search?q=covid+data
# We will try to merge the information from different dataset and come up with updated Covid cases nd Covid Death info country wise.


# Loading the different data frames
import pandas as pd
from pandas.conftest import ordered

# Define the path to your .csv file
file_path = 'covid_dataset.csv'
file_path_cases = 'weekly-covid-cases.csv'
file_path_death = 'weekly-covid-deaths.csv'

# Load the data into a pandas DataFrame
try:
    # Reading the file
    df = pd.read_csv(file_path)
    df_cases = pd.read_csv(file_path_cases)
    df_death = pd.read_csv(file_path_death)
    print("Data loaded successfully!")
    # Reading first 10 rows
    #print(df.head(10)) # Print the first 10 rows of the DataFrame
except FileNotFoundError:
    print(f"Error: The file at {file_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")






Data loaded successfully!


## Step 1 To check on the dataframe quality and check on Outliers


In [19]:
# In order to get fresh data and we need to update our data frame
# We will merge the cases reported and Deaths in One data Frame

# Lets start with cleaning up the dataframes of Death and Covid Cases
# The Dataframe.describe() shows data in Exponential so changing that
pd.set_option('display.float_format', '{:f}'.format)

# Checking the Range of the data to see if there are outliers
print(df_death.describe())

# Max No of Deaths is about 100,000 which is seems to be correct and doens't seems to be an Outlier

print(df_cases.describe())

# On seeing case data it seems to be an outlier. We will examine the data and see if that is required or is an outlier


# --- 1. Sort the DataFrame ---
# Sort by 'Score' in descending order (highest score first)
df_sorted = df_cases.sort_values(by='Weekly cases', ascending=False)

# --- 2. Select the top 10 rows ---
# Use the .head(10) method on the sorted DataFrame
top_10_rows = df_sorted.head(10)
print (top_10_rows)

# The data seems to be a consolidated number , of World, which is not required for our project so we can eliminate it.
# Also we found the data contains some continent level info which will not be required for our analysis and could be eliminated

# All World level and continent and group of countries information has to be eliminated.

unique_value =df_cases['Entity'].unique()
print(unique_value)

values_to_drop = ['World','World excl. China','World excl. China and South Korea','World excl. China, South Korea, Japan and Singapore','Upper-middle-income countries','Oceania','North America','South America','Lower-middle-income countries','Low-income countries','High-income countries','Europe','European Union (27)','Asia','Asia excl. China','Africa']

df_cases = df_cases[~df_cases['Entity'].isin(values_to_drop)]
df_death = df_death[~df_death['Entity'].isin(values_to_drop)]


       Weekly deaths
count  517936.000000
mean      703.079655
std      5172.164055
min         0.000000
25%         0.000000
50%         0.000000
75%        13.000000
max    103745.000000
         Weekly cases
count   517028.000000
mean     73585.883828
std     699715.312594
min          0.000000
25%          0.000000
50%         31.000000
75%       1353.000000
max   44814230.000000
                               Entity      Code         Day  Weekly cases
503481                          World  OWID_WRL  2022-12-26      44814230
503482                          World  OWID_WRL  2022-12-27      44236932
503480                          World  OWID_WRL  2022-12-25      44236205
26160                            Asia       NaN  2022-12-26      42768774
503483                          World  OWID_WRL  2022-12-28      42712671
26161                            Asia       NaN  2022-12-27      42222120
26159                            Asia       NaN  2022-12-25      42142424
503479               

## Step 2 To check on the dataframe duplicates and NaN


In [20]:


# --- Code to count duplicates ---
duplicate_count_cases = df_cases.duplicated().sum()
print(f"Total number of duplicate rows found: {duplicate_count_cases}")

# --- Code to count duplicates ---
duplicate_death_cases = df_death.duplicated().sum()
print(f"Total number of duplicate rows found: {duplicate_death_cases}")

# No Duplicates in the DataFrame
#df_death.drop_duplicates()
#df_cases.drop_duplicates()

# --- Code to count NaN ---
null_count_cases = df_cases.isna().sum().sum()
print(f"Total number of Null rows found: {null_count_cases}")

# --- Code to count NaN ---
null_death_cases = df_death.isna().sum().sum()
print(f"Total number of Null rows found: {null_death_cases}")

# Checking the Rows having NaN
null_rows_any_cases = df_cases[df_cases.isnull().any(axis=1)]
print(null_rows_any_cases)

# Checking the Rows having NaN
null_rows_any_deaths = df_death[df_death.isnull().any(axis=1)]
print(null_rows_any_deaths)



# On Checking the data it is found that there are rows which has Africa continent info as well as Asia except China as different categories. For our analysis we will only take countries and excluded continent and non countries info
# drop NaN from data frame
df_cases.dropna(inplace=True)
df_death.dropna(inplace=True)

print(df_cases.head(10))
print(df_death.head(10))



Total number of duplicate rows found: 0
Total number of duplicate rows found: 0
Total number of Null rows found: 0
Total number of Null rows found: 0
Empty DataFrame
Columns: [Entity, Code, Day, Weekly cases]
Index: []
Empty DataFrame
Columns: [Entity, Code, Day, Weekly deaths]
Index: []
        Entity Code         Day  Weekly cases
0  Afghanistan  AFG  2020-01-09             0
1  Afghanistan  AFG  2020-01-10             0
2  Afghanistan  AFG  2020-01-11             0
3  Afghanistan  AFG  2020-01-12             0
4  Afghanistan  AFG  2020-01-13             0
5  Afghanistan  AFG  2020-01-14             0
6  Afghanistan  AFG  2020-01-15             0
7  Afghanistan  AFG  2020-01-16             0
8  Afghanistan  AFG  2020-01-17             0
9  Afghanistan  AFG  2020-01-18             0
        Entity Code         Day  Weekly deaths
0  Afghanistan  AFG  2020-01-09              0
1  Afghanistan  AFG  2020-01-10              0
2  Afghanistan  AFG  2020-01-11              0
3  Afghanistan  A

## Step 3 To reduce the data frame size by reporting cases and deaths Monthly instead of Daily. and merge the two Data Frames

In [21]:

# Inorder to reduce the records, we will sum the data month wise for each data frame
df_cases['Day'] = pd.to_datetime(df_cases['Day'])

# Extract Year and Month into new columns
df_cases['Year'] = df_cases['Day'].dt.year
df_cases['Month'] = df_cases['Day'].dt.month

# Inorder to reduce the records, we will sum the data month wise for each data frame
df_death['Day'] = pd.to_datetime(df_death['Day'])

# Extract Year and Month into new columns
df_death['Year'] = df_death['Day'].dt.year
df_death['Month'] = df_death['Day'].dt.month

# Define the list of columns to group by
grouping_keys = ['Year', 'Month', 'Code', 'Entity']

df_grouped_deaths = df_death.groupby(grouping_keys)['Weekly deaths'].sum().reset_index()
df_grouped_cases = df_cases.groupby(grouping_keys)['Weekly cases'].sum().reset_index()

df_merged_same_1 = pd.merge(
    df_grouped_deaths,
    df_grouped_cases,
    on=['Code', 'Year','Month', 'Entity'],
    how='inner'
)
df_merged_same_1= df_merged_same_1.sort_values(by=['Entity'])

# We want to report the data as on start of each month and later the weather will be pulled from API for start of each month.
# Using Month and Year will form the start of the month
df_merged_same_1['Date_String'] = (
    df_merged_same_1['Year'].astype(str) + '-' +
    df_merged_same_1['Month'].astype(str).str.zfill(2) + '-01'
)

df_merged_same_1['New_Date'] = pd.to_datetime(df_merged_same_1['Date_String'])
df_merged_same_1.info()
df_merged_same_1.to_csv('output_data1.csv', index=False)
df_information=df_merged_same_1


<class 'pandas.core.frame.DataFrame'>
Index: 15971 entries, 4195 to 15970
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Year           15971 non-null  int32         
 1   Month          15971 non-null  int32         
 2   Code           15971 non-null  object        
 3   Entity         15971 non-null  object        
 4   Weekly deaths  15971 non-null  int64         
 5   Weekly cases   15971 non-null  int64         
 6   Date_String    15971 non-null  object        
 7   New_Date       15971 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(2), int64(2), object(3)
memory usage: 998.2+ KB


# Step 4 - Get the Primary Data frame, remove duplicates and countries with state specific information. Only add the state national capital region for big countries which had state level informations

In [22]:

df_multi_dropped = df.drop(columns=['day', 'temperature','humidity','confirmed','deaths','recovered','quarantine', 'restrictions','schools'])
df_multi_dropped = df_multi_dropped.drop_duplicates()
df_multi_dropped.head(10)
column_name = 'Country/Region'
# Country/Region

duplicate_rows = df_multi_dropped[df_multi_dropped[column_name].duplicated(keep=False)]

print(f"\n--- All Rows Containing Duplicated Values in '{column_name}' ---")
print(duplicate_rows.sort_values(by=column_name))



# ON analysis it is found that Australia, Canada, USA and China has state level info in the Data frame, SO we will take only one Geaographical Record for the Country (their Capital)


province_to_select =['Australian Capital Territory','Maryland','Beijing','Ontario']

# Condition: Select rows where 'Country/Region' is above Province list
big_countries_data = df_multi_dropped[df_multi_dropped['Province/State'].isin(province_to_select)]

# dropping multiple rows of the states of four countries
values_to_drop = ['Australia','China','US','Canada']

df_multi_dropped = df_multi_dropped[~df_multi_dropped['Country/Region'].isin(values_to_drop)]
df_multi_dropped.drop_duplicates().to_csv('output_data.csv', index=False)

column_to_check = 'Province/State'
df_not_null = df_multi_dropped[df_multi_dropped[column_to_check].notna()]
print(df_not_null)

# There are few places like Greenland which shows under Denmark, Correcting those to come under Countries instead of Provinces

df_multi_dropped['Country/Region'] = df_multi_dropped['Province/State'].where(
    df_multi_dropped['Province/State'].notna(), # Condition: Keep value if Target_Column is NOT NaN
    other=df_multi_dropped['Country/Region']    # Else (if it IS NaN), replace with Source_Column
)



#print(big_countries_data.tail(15))


#df_multi_dropped.drop_duplicates().to_csv('output_data.csv', index=False)

df_master= df_multi_dropped.drop_duplicates()
df_master = pd.concat([df_multi_dropped, big_countries_data], ignore_index=True)
print(df_master.head(15))
df_master.drop_duplicates().to_csv('output_data.csv', index=False)



--- All Rows Containing Duplicated Values in 'Country/Region' ---
                     Province/State  Country/Region        lat       long  \
488    Australian Capital Territory       Australia -35.473500 149.012400   
549           From Diamond Princess       Australia  35.443700 139.638000   
610                 New South Wales       Australia -33.868800 151.209300   
671              Northern Territory       Australia -12.463400 130.845600   
732                      Queensland       Australia -28.016700 153.400000   
...                             ...             ...        ...        ...   
17568                    Montserrat  United Kingdom  16.742500 -62.187400   
17324                Cayman Islands  United Kingdom  19.313300 -81.254600   
17263                       Bermuda  United Kingdom  32.307800 -64.750500   
17385               Channel Islands  United Kingdom  49.372300  -2.364400   
17629                United Kingdom  United Kingdom  55.378100  -3.436000   

        

# Step 5 - Merge the data frame using Fuzzy logic

In [23]:
from fuzzywuzzy import fuzz

# --- Step 1: Prepare for Fuzzy Match (Cartesian Product) ---
# Create a temporary DataFrame of all possible matches by merging on a dummy key.
# This assumes the combined size is manageable (e.g., L_rows * R_rows < 1,000,000).
df_master['_key'] = 0
df_information['_key'] = 0
df_cartesian = pd.merge(df_master, df_information, on='_key').drop('_key', axis=1)



# --- Step 2: Calculate Similarity Score ---
def get_fuzzy_score(row):
    """Calculates the partial ratio score between the two product names."""
    # Using token_set_ratio is often better for dealing with word order/punctuation
    return fuzz.token_set_ratio(row['Country/Region'], row['Entity'])

# Apply the function to create a new column with the score
df_cartesian['score'] = df_cartesian.apply(get_fuzzy_score, axis=1)


# --- Step 3: Filter and Select the Best Match ---
# Define a minimum threshold for a match
THRESHOLD = 80

# Filter out rows below the threshold
df_matches = df_cartesian[df_cartesian['score'] >= THRESHOLD]

# For each left product, keep only the right product with the highest score
idx = df_matches.groupby(['Country/Region'])['score'].transform("max") == df_matches['score']
df_best_matches = df_matches[idx]
print(df_best_matches)
df_best_matches.loc[:,'temperature'] = ""
df_best_matches.loc[:,'feels_like'] = ""
df_best_matches.loc[:,'humidity'] = ""

df_best_matches.drop_duplicates().to_csv('output_data2.csv', index=False)


KeyboardInterrupt: 

* What changes were made to the data?

The data was modified to include data for larger duration which was earlier 62 days. Now it contains data from 01/2020 - 09/2025. This will help show us the effect of weather on the Covid trackers properly. For the countries which are bigger like (Canada, China, USA), a separate statewise analysis is required and wont be accurate because the weather is very different throughout of the size. For sake of data we just have considered the National Capital of such countries.


* Are there any legal or regulatory guidelines for your data or project topic?

Since the data doesn't contain the private Health info it doesn't violate and personal Health privacy violations, though ethics and reasearch guidelines should be maintained. Transparency and Mitigation of Algorithmic Bias to ensure the models and findings do not unfairly impact certain communities.



* What risks could be created based on the transformations done?

Duplication of data might happen in case of joining multiple dataframes, Fuzzy algorithm threshold has to be watched in order to match the appropriate keys.


* Did you make any assumptions in cleaning/transforming the data?
For bigger countires the weather linking might not work until looked at state level. For big countries the analysis should be handled separately


* How was your data sourced/verified for credibility?

My response capability is based on the information I was trained on (a massive dataset of text and code) and, for current or specific inquiries, the information I can retrieve using my Google Search tool. I do not have a private, project-specific dataset of COVID-19 that I have sourced or verified.
However I am relying on https://ourworldindata.org which sources data from Authentic sources


* Was your data acquired in an ethical way?

It was an open source data for public use.


* How would you mitigate any of the ethical implications you have identified?

I'd mitigate ethical risks by focusing on two areas: Privacy and Fairness.

For Privacy, I'd use aggressive data aggregation (county-level, weekly counts) and implement Data Use Agreements (DUAs) to prohibit re-identification attempts. Data would be stored in a secure, encrypted data enclave.

For Fairness, I'd explicitly include and control for socioeconomic factors in the analysis to prevent weather correlations from masking underlying health disparities. All model results would be reported with clear limitations and a focus on actionable public health policy, not stigmatization.